## Implementing a Birth-Death model of Speciation and Extinction

Simple models are extremely useful in bioinformatics. Although a simple model will not generally capture all aspects of a biological process, it can provide a very useful baseline: if available biological data is well explained by the simple model, then no further explanation is demanded. On the other hand, if the simple model does a poor job of explaining the observed data, then further investigations, and perhaps a more complex model may be justified. It is by digging in deeply to such cases - where something 'weird' or surprising is going on - that many interesting discoveries in biology have been uncovered

A simple way to model the evolution of species diversity is through a birth-death process. In such a process, species diverge (forming two new species) or go extinct at some rate. In this section we will implement a birth-death model of species diversity using our PhyloNode object.

In our model we start with one initial species representing the common ancestor of the tree. New species form by speciation events which cause two child species to appear. Species can also be removed by extinction events. We can then add a chance of speciating or going extinct at each point in time.

To do so we'll have to add a speciation function that automatically splits a lineage in two, a property to keep track of which nodes are extinct, and an update function that updates a node on the tree, causing it to possibly speciate or go extinct.

Finally, outside of our class we'll want to write a for loop to run the simulation for a given number of iterations and report the results.

In [4]:
from random import random, choice

class PhyloNode():
    """A node on a phylogenetic tree"""

    def __init__(self,children=None,parent=None,\
      name = None):
      """Initiate a node on a phylogenetic tree
      children -- a list of PhyloNode objects
        descending immediately from this node
      name -- a string for the name of the node
      parent -- a single PhyloNode object for the parent
      """
      self.Name = name
      self.Children = []
      if children:
        self.Children.extend(children)
      self.Parent = parent
      self.Extinct = False

    def isTip(self):
      """Return True if the node is a tip"""
      if not self.Children: #capture None or []
        return True
      else:
        return False

    def isRoot(self):
      """Return True if the node is the root of the whole tree"""
      if not self.Parent:
        return True
      else:
        return False

    def getDescendants(self):
        """Return a list of PhyloNodes descending from the current node"""

        if self.isTip():
            print(self.Name," is a tip ... returning []")
            return []

        descendants = self.Children or []
        for c in self.Children:
            #The set of descendants is described
            #by the descendants of all the nodes
            #immediate children.
            if not c.isTip():
                child_descendants = c.getDescendants()
                descendants.extend([c for c in child_descendants if c not in descendants])

            #Side note: this will fail on enormous trees
            #due to the recursion limit. Not normally a problem though.
        return descendants

    def getAncestors(self):
        """Return the ancestors of the given node"""
        if self.isRoot():
            return None

        ancestors = [self.Parent]
        parents_ancestors = self.Parent.getAncestors()
        if parents_ancestors:
            ancestors.extend(parents_ancestors)
        return ancestors

    def getRoot(self):
        """Return the root node"""
        curr_node = self
        while not curr_node.isRoot():
            curr_node = curr_node.Parent
        return curr_node

    def addChild(self,child):
        """Attach a child node"""
        if child not in self.Children:
            self.Children.append(child)
        child.Parent = self
 
    def addParent(self,parent):
        """Attach a parent node"""
        self.Parent = parent
        parent.Children.append(self)
    
    def getMRCA(self,other):
        """Return PhyloNode for the most recent common ancestor with other
        other -- another PhyloNode object in the same tree
        """
        most_recent_common_ancestor = None
        #Cache the ancestors of other since
        #we'll refer to it often
        other_ancestors = other.getAncestors()
        self_ancestors = self.getAncestors()
        #First check for the trivial case
        #where one node is root (but be sure they're on the same tree)
        if self.isRoot() and self in other_ancestors:
            return self

        if other.isRoot() and other in self_ancestors:
            return other

        for a in self_ancestors:
            #Note these will be in order of relatedness
            if a in other_ancestors:
                #End the loop the first time this happens
                #since we want the most
                #recent common ancestor
                most_recent_common_ancestor = a
                break

        if not most_recent_common_ancestor:
            raise ValueError("No common ancestor found for ",self.Name," and ",other.Name,\
            ". Are they on the same tree?")

        return most_recent_common_ancestor
    
    def speciate(self):
        """Add two children descending from this node"""
        if self.Children:
            raise ValueError("Internal nodes can't speciate")
        if self.Extinct:
            raise ValueError("Extinct nodes can't speciate")

        child1_name = self.Name + "A"
        child1=PhyloNode(name=child1_name)
        self.addChild(child1)
        child2_name = self.Name + "B"
        child2=PhyloNode(name=child2_name)
        self.addChild(child2)


    def update(self,speciation_chance=0.25,\
        extinction_chance=0.25):
        """Update the node by speciation or extinction"""
        if not self.isTip():
            print("Not updating node {name} - it's not a tip".format(name=self.Name))
            return None

        if random() < extinction_chance:
            print(self.Name," goes extinct!")
            self.Extinct = True

        if self.Extinct:
            print("Not updating node {name} - it's extinct".format(name=self.Name))
            return None
        print("Updating node:{name}".format(name=self.Name))

        if random() < speciation_chance:
            #Speciate
            print("Node {name} speciates!".format(name=self.Name))

## Running the birth-death model

Now that we have a way to update the tree at each timestep, we can build a simple tree and use a for-loop to run the birth-death model in a loop.

In [5]:

#Build the tree!
root = PhyloNode(name="root")
A = PhyloNode(name="A")
B = PhyloNode(name="B")
root.addChild(A)
root.addChild(B)

time = 5
#Demo our tree functions
print("Root node name:",root.Name)

for t in range(1,time):
    print("="*80)
    print("Beginning simulation of time:",t)
    print("="*80)
    #I want to keep the original set of nodes to update
    #so that new species don't immediately get simulated
    nodes_to_update = [n for n in root.getDescendants()]
    for node in nodes_to_update:

        if not node.isTip():
            continue
        print("About to update node:",node.Name)
        node.update()
    print("Nodes in tree at time ",t,":",len([n.Name for n in root.getDescendants()]))
    print("Tree tips:",[n.Name for n in root.getDescendants() if n.isTip()])

Root node name: root
Beginning simulation of time: 1
About to update node: A
A  goes extinct!
Not updating node A - it's extinct
About to update node: B
Updating node:B
Nodes in tree at time  1 : 2
Tree tips: ['A', 'B']
Beginning simulation of time: 2
About to update node: A
A  goes extinct!
Not updating node A - it's extinct
About to update node: B
Updating node:B
Nodes in tree at time  2 : 2
Tree tips: ['A', 'B']
Beginning simulation of time: 3
About to update node: A
Not updating node A - it's extinct
About to update node: B
Updating node:B
Nodes in tree at time  3 : 2
Tree tips: ['A', 'B']
Beginning simulation of time: 4
About to update node: A
A  goes extinct!
Not updating node A - it's extinct
About to update node: B
Updating node:B
Nodes in tree at time  4 : 2
Tree tips: ['A', 'B']


## Simulating Trait Evolution

We can add traits to our birth-death model of tree evolution by adding a dict of BinaryTraits to each tree node, and then modifying it as necessary over time. We need to set up the __init__ method of the PhyloNode object to accept trait information. We can then pass on the traits of ancestral organisms to their descendants when nodes speciate.

In [7]:
from copy import deepcopy
from random import random,choice

class PhyloNode():
    """A node on a phylogenetic tree"""

    def __init__(self,children=None,parent=None,\
      name = None,binary_traits=None):
      """Initiate a node on a phylogenetic tree
      children -- a list of PhyloNode objects
        descending immediately from this node
      name -- a string for the name of the node
      parent -- a single PhyloNode object for the parent
      binary_traits -- a dict of True/False traits
      """
      self.Name = name
      self.Children = []
      if children:
        self.Children.extend(children)
      self.Parent = parent
      self.BinaryTraits = deepcopy(binary_traits) or {}
      self.Extinct = False

    def isTip(self):
      """Return True if the node is a tip"""
      if not self.Children: #capture None or []
        return True
      else:
        return False

    def isRoot(self):
      """Return True if the node is the root of the whole tree"""
      if not self.Parent:
        return True
      else:
        return False

    def getDescendants(self):
        """Return a list of PhyloNodes descending from the current node"""

        if self.isTip():
            print(self.Name," is a tip ... returning []")
            return []

        descendants = self.Children or []
        for c in self.Children:
            #The set of descendants is described
            #by the descendants of all the nodes
            #immediate children.
            if not c.isTip():
                child_descendants = c.getDescendants()
                descendants.extend([c for c in child_descendants if c not in descendants])

            #Side note: this will fail on enormous trees
            #due to the recursion limit. Not normally a problem though.
        return descendants

    def getAncestors(self):
        """Return the ancestors of the given node"""
        if self.isRoot():
            return None

        ancestors = [self.Parent]
        parents_ancestors = self.Parent.getAncestors()
        if parents_ancestors:
            ancestors.extend(parents_ancestors)
        return ancestors

    def getRoot(self):
        """Return the root node"""
        curr_node = self
        while not curr_node.isRoot():
            curr_node = curr_node.Parent
        return curr_node

    def addChild(self,child):
        """Attach a child node"""
        if child not in self.Children:
            self.Children.append(child)
        child.Parent = self
 
    def addParent(self,parent):
        """Attach a parent node"""
        self.Parent = parent
        parent.Children.append(self)
    
    def getMRCA(self,other):
        """Return PhyloNode for the most recent common ancestor with other
        other -- another PhyloNode object in the same tree
        """
        most_recent_common_ancestor = None
        #Cache the ancestors of other since
        #we'll refer to it often
        other_ancestors = other.getAncestors()
        self_ancestors = self.getAncestors()
        #First check for the trivial case
        #where one node is root (but be sure they're on the same tree)
        if self.isRoot() and self in other_ancestors:
            return self

        if other.isRoot() and other in self_ancestors:
            return other

        for a in self_ancestors:
            #Note these will be in order of relatedness
            if a in other_ancestors:
                #End the loop the first time this happens
                #since we want the most
                #recent common ancestor
                most_recent_common_ancestor = a
                break

        if not most_recent_common_ancestor:
            raise ValueError("No common ancestor found for ",self.Name," and ",other.Name,\
            ". Are they on the same tree?")

        return most_recent_common_ancestor
    
    def speciate(self):
        """Add two children descending from this node"""
        if self.Children:
            raise ValueError("Internal nodes can't speciate")
        if self.Extinct:
            raise ValueError("Extinct nodes can't speciate")

        child1_name = self.Name + "A"
        child1=PhyloNode(name=child1_name,binary_traits=self.BinaryTraits)
        self.addChild(child1)
        child2_name = self.Name + "B"
        child2=PhyloNode(name=child2_name,binary_traits=self.BinaryTraits)
        self.addChild(child2)


    def update(self,speciation_chance=0.25,trait_change_chance=0.25,\
        extinction_chance=0.25):
        """Update the node by speciation, extinction or trait gain or loss"""
        if not self.isTip():
            print("Not updating node {name} - it's not a tip".format(name=self.Name))
            return None

        if random() < extinction_chance:
            print(self.Name," goes extinct!")
            self.Extinct = True

        if self.Extinct:
            print("Not updating node {name} - it's extinct".format(name=self.Name))
            return None
        print("Updating node:{name}".format(name=self.Name))


        for trait,value in self.BinaryTraits.items():
            if random() < trait_change_chance:
                self.BinaryTraits[trait] = not self.BinaryTraits[trait]
                print(self.Name," has a new trait value for ",\
                  trait,": ",self.BinaryTraits[trait],"!")

        if random() < speciation_chance:
            #Speciate
            print("Node {name} speciates!".format(name=self.Name))
            self.speciate()

Now let's use our updated, trait-aware PhyloNode object to simulate tree evolution:

In [12]:
#Define traits
binary_traits = {"Blubber":False,"Red coat":True,"Big Ears":False,\
  "Horn":False,"Wings":False}
#Build the tree!
root = PhyloNode(name="root",binary_traits=binary_traits)
A = PhyloNode(name="A",binary_traits=binary_traits)
B = PhyloNode(name="B",binary_traits=binary_traits)
root.addChild(A)
root.addChild(B)

time = 5
#Demo our tree functions
print("Root node name:",root.Name)

for t in range(1,time):
    print("="*80)
    print("Beginning simulation of time:",t)
    print("="*80)
    #I want to keep the original set of nodes to update
    #so that new species don't immediately get simulated
    nodes_to_update = [n for n in root.getDescendants()]
    for node in nodes_to_update:

        if not node.isTip():
            continue
        print("About to update node:",node.Name)
        node.update()
    print("Nodes in tree at time ",t,":",len([n.Name for n in root.getDescendants()]))
    print("Tree tips:",[n.Name for n in root.getDescendants() if n.isTip()])


Root node name: root
Beginning simulation of time: 1
About to update node: A
Updating node:A
About to update node: B
B  goes extinct!
Not updating node B - it's extinct
Nodes in tree at time  1 : 2
Tree tips: ['A', 'B']
Beginning simulation of time: 2
About to update node: A
Updating node:A
A  has a new trait value for  Horn :  True !
About to update node: B
B  goes extinct!
Not updating node B - it's extinct
Nodes in tree at time  2 : 2
Tree tips: ['A', 'B']
Beginning simulation of time: 3
About to update node: A
Updating node:A
A  has a new trait value for  Red coat :  False !
A  has a new trait value for  Big Ears :  True !
A  has a new trait value for  Horn :  False !
About to update node: B
Not updating node B - it's extinct
Nodes in tree at time  3 : 2
Tree tips: ['A', 'B']
Beginning simulation of time: 4
About to update node: A
A  goes extinct!
Not updating node A - it's extinct
About to update node: B
B  goes extinct!
Not updating node B - it's extinct
Nodes in tree at time  4 

## Exercises

**Exercise 1**. Quantify the range of likely species diversity for given speciation/extinction rates

Rerun the simulation many times. Each time, count up the number of surviving tree tips by time 5. You'll notice it's not always the same number! The number of surviving species (tips) is a measure of the range of species diversity we might observe in a particular taxon with the speciation and extinction rates we specified up above. What are the minimum,maximum,mean, and standard deviation for species diversity with the parameters given up above?  Now try changing the speciation or extinction rate. How does that change species diversity?

*Hint*: it will be less laborious to do this is you make a for loop to run the simulation many times and count up results.

*Hint 2*: if you like, you can also plot the results in a histogram. See for example how to do so in matplotlib (a graphing library that comes with Anaconda python): [histogram example](https://matplotlib.org/3.1.1/gallery/statistics/hist.html)

**Exercise 2** Add continuous traits. Modify our birth-death simulation to allow for continuous traits. You will likely want to define a new class property called self.ContinuousTraits that holds a dict with keys equal to the names of continuous traits, and their values represented by floats. Trait changes will be represented by modifying the trait value. You will have to make some subjective choices, but that is OK. However, do be careful to record which choices you make and consider what assumptions, if any, they encode about the process of evolution for your traits. Examples include whether you modify the value of your trait by addition vs. multiplication,  and whether you want to bound your traits to values >=0. 

**Exercise 3**. Allow for unequal rates of evolution. Our birth-death simulation as written currently assumes that all traits evolve at equal rates. Modify it to allow for each trait to evolve at a different rate (e.g. to have a different chance of changing on each branch). *Hint*: you may wish to store the rate of evolution of each trait in a dict.

## Project Ideas

#### How do the rates of speciation, extinction, and trait change affect the occurence of homology vs. convergent evolution?

*Rationale*: When species share a trait, a fundamental evolutionary question is whether that trait arose from a common evolutionary origin or if the trait evolved separately in the lineages leading to the two species. Presumably, rates of speciation vs. rates of trait change should affect how often shared traits are homologous vs. convergently evolved. Here are some example predictions one might make:

* if speciation is fast and trait change is slow, many species will tend to share common trait values that they inherit from their common ancestor (homologous traits)
* if trait change is extremely fast and speciation is slow, even sister taxa may have essentially random trait values and nearly all shared traits will be convergently evolved.

*Formal problem* For a tree with N nodes and a single binary trait, use simulation to find the probability that two random nodes on the tree share the trait by homology. Also find the probability that two random nodes on the tree share the trait by convergent evolution. How does the ratio of these probabilities change with varying sizes of tree, and rates of speciation, extinction, and trait change?

*Relevant methods*. Testing the probability above is a form of Monte Carlo method. Reviewing or learning for the first time about Monte Carlo methods (e.g. in the statistics section of this text) may provide useful background.